# **Signa_Lab ITESO: Generación de relaciones semánticas (*embeddings*)**

## **Proyecto:** *Selección de Preguntas de Redes Sociales para 1er Debate Presidencial INE 2024 (Formato A)*

Cuaderno de código para generar *embeddings* (relaciones semánticas) con las preguntas de la ciudadanía de cada uno de los 6 temas en la muestra estratificada previamente elaborada, utilizando el modelo de lenguaje de software libre *intfloat/multilingual-e5-large-instruct* (Wang et al, 2024) y la librería de aprendizaje profundo *sentence-transformers* (Reimers & Gurevych, 2019).

## 1. Importar librerías, dependencias y archivo de datos a para la generación de *embeddings*


**Instalar librerías:**

In [ ]:
# Instalar librerías de Python necesarias

!pip install tensorflow_text
!pip install pandas
!pip install sentence_transformers
!pip install cohere
!pip install numpy
!pip install operator
!pip install torch
!pip install tqdm
!pip install transformers
!pip install tensorflow

**Importar librerías:**

In [ ]:
# Importar librerías de Python necesarias

import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import operator
from tqdm import tqdm
import time
import torch
from transformers import BertForMaskedLM, BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf

In [3]:
# Definir función para implementar barra de progreso sobre el procesamiento
def with_progress_bar(func):
    def wrapper(*args, **kwargs):
        # Inicia el contador de tiempo
        start_time = time.time()

        # Crea una barra de progreso
        pbar = tqdm(total=100, desc="Processing", ncols=70)

        # Define una función interna para actualizar la barra de progreso
        def update_progress_bar(i):
            pbar.update(i)

        # Agrega la función de actualización al diccionario de argumentos
        kwargs['update_progress_bar'] = update_progress_bar

        # Ejecuta la función original
        result = func(*args, **kwargs)

        # Finaliza la barra de progreso
        pbar.close()

        # Calcula y muestra el tiempo de ejecución
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

        return result
    return wrapper

**Cargar modelo de lenguaje** para su implementación en librería de *sentence-transformers*:

In [ ]:
# Indicar ruta y nombre de modelo de lenguaje a cargar en librería de sentence tranformers

# ejemplo de ruta de modelo local:
modelo = "./modelos/multilingual-e5-large-instruct"

# ejemplo de ruta de modelo en la nube desde Hugging Face:
# modelo = "intfloat/multilingual-e5-large-instruct"

# cargar modelo para embeddings
embedder = SentenceTransformer(modelo)

In [5]:
embedder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

**Indicar rutas y cargar archivos de datos** de preguntas por tema para generar relaciones semánticas:

In [ ]:
# Rutas locales de archivos de Excel segmentados por cada tema de la muestra estratificada:
rutaCorrupcion = "data/20240325_1759_PreguntasDebateINE_MuestraEstratificada_Corrupcion.xlsx"
rutaEducacion = "data/20240325_1759_PreguntasDebateINE_MuestraEstratificada_Educacion.xlsx"
rutaNoDiscriminacion = "data/20240325_1759_PreguntasDebateINE_MuestraEstratificada_NoDiscriminacion.xlsx"
rutaSalud = "data/20240325_1759_PreguntasDebateINE_MuestraEstratificada_Salud.xlsx"
rutaTransparencia = "data/20240325_1759_PreguntasDebateINE_MuestraEstratificada_Transparencia.xlsx"
rutaViolenciaMujeres = "data/20240325_1759_PreguntasDebateINE_MuestraEstratificada_ViolenciaMujeres.xlsx"

# extraer nombres de archivos como referencia
nombreArchivoCorrupcion = rutaCorrupcion.split("/")[-1].split('.')[0]
nombreArchivoEducacion = rutaEducacion.split("/")[-1].split('.')[0]
nombreArchivoNoDiscriminacion = rutaNoDiscriminacion.split("/")[-1].split('.')[0]
nombreArchivoSalud = rutaSalud.split("/")[-1].split('.')[0]
nombreArchivoTransparencia = rutaTransparencia.split("/")[-1].split('.')[0]
nombreArchivoViolenciaMujeres = rutaViolenciaMujeres.split("/")[-1].split('.')[0]

# leer datos importados (formato Excel)
dfCorrupcion = pd.read_excel(rutaCorrupcion)
dfEducacion = pd.read_excel(rutaEducacion)
dfNoDiscriminacion = pd.read_excel(rutaNoDiscriminacion)
dfSalud = pd.read_excel(rutaSalud)
dfTransparencia = pd.read_excel(rutaTransparencia)
dfViolenciaMujeres = pd.read_excel(rutaViolenciaMujeres)

In [7]:
# Limpiar columnas adicionales
dfCorrupcion = dfCorrupcion.drop('Column', axis=1)
dfEducacion = dfEducacion.drop('Column', axis=1)
dfNoDiscriminacion = dfNoDiscriminacion.drop('Column', axis=1)
dfSalud = dfSalud.drop('Column', axis=1)
dfTransparencia = dfTransparencia.drop('Column', axis=1)
dfViolenciaMujeres = dfViolenciaMujeres.drop('Column', axis=1)

**Revisar datos importados:**

Preguntas en muestra estratificada sobre **Combate a la Corrupción**

In [ ]:
# Previsualizar tabla de datos importados
dfCorrupcion.head()

In [9]:
# Verificar número de filas y columnas en datos importados del tema
dfCorrupcion.shape

(389, 12)

Preguntas en muestra estratificada sobre **Educación**:

In [ ]:
# Previsualizar tabla de datos importados
dfEducacion.head()

In [11]:
# Verificar número de filas y columnas en datos importados del tema
dfEducacion.shape

(391, 12)

Preguntas en muestra estratificada sobre **No Discriminación y Grupos Vulnerables:**

In [ ]:
# Previsualizar tabla de datos importados
dfNoDiscriminacion.head()

In [13]:
# Verificar número de filas y columnas en datos importados del tema
dfNoDiscriminacion.shape

(193, 12)

Preguntas en muestra estratificada sobre **Salud**:

In [ ]:
# Previsualizar tabla de datos importados
dfSalud.head()

In [15]:
# Verificar número de filas y columnas en datos importados del tema
dfSalud.shape

(353, 12)

Preguntas en muestra estratificada sobre **Transparencia:**

In [ ]:
# Previsualizar tabla de datos importados
dfTransparencia.head()

In [17]:
# Verificar número de filas y columnas en datos importados del tema
dfTransparencia.shape

(219, 12)

Preguntas en muestra estratificada sobre **Violencia en Contra de las Mujeres:**

In [ ]:
# Previsualizar tabla de datos importados
dfViolenciaMujeres.head()

In [19]:
# Verificar número de filas y columnas en datos importados del tema
dfViolenciaMujeres.shape

(156, 12)

## 2. Generar *embeddings* con el modelo de lenguaje cargado

**Definir función para generar embeddings:**

In [20]:
# Definir función para generar embeddings
@with_progress_bar
def generate(mod, dfl, colText, update_progress_bar=None):
    tot = len(dfl)
    dfW = dfl
    dfW.loc[:, "Embedding"] = None
    for index, row in dfW.iterrows():
        # Iterar sobre el DataFrame y codificar cada texto
        embedding = mod.encode(str(row[colText])).tolist()
        dfW.at[index, 'Embedding'] = embedding
        # Actualizar la barra de progreso una vez por cada 10 filas
        if update_progress_bar is not None:
            update_progress_bar((index/tot)*2)
    return dfW

**Generar embeddings con datos de cada tema y modelo cargados:**

In [21]:
# Combate a la Corrupción
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddingsCorrupcion = generate(embedder, dfCorrupcion, "sem_text")

Processing:  99%|████▉| 99.25964010282775/100 [00:35<00:00,  5.06it/s]/home/estacion/.local/lib/python3.10/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Processing: 387.9999999999999it [01:07,  5.76it/s]                    

Tiempo de ejecución: 67.34 segundos


In [22]:
# Educación
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddingsEduacion = generate(embedder, dfEducacion, "sem_text")

Processing: 390.00000000000017it [00:55,  6.98it/s]                   

Tiempo de ejecución: 55.90 segundos


In [23]:
# No Discriminación y Grupos Vulnerables
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddingsNoDiscriminacion = generate(embedder, dfNoDiscriminacion, "sem_text")

Processing: 192.00000000000017it [00:28,  6.81it/s]                   

Tiempo de ejecución: 28.19 segundos


In [24]:
# Salud
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddingsSalud = generate(embedder, dfSalud, "sem_text")

Processing: 351.99999999999955it [00:50,  6.97it/s]                   

Tiempo de ejecución: 50.47 segundos


In [25]:
# Transparencia
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddingsTransparencia = generate(embedder, dfTransparencia, "sem_text")

Processing: 218.00000000000003it [00:31,  6.90it/s]                   

Tiempo de ejecución: 31.60 segundos


In [26]:
# Violencia en Contra de las Mujeres
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddingsViolenciaMujeres = generate(embedder, dfViolenciaMujeres, "sem_text")

Processing: 154.99999999999997it [00:23,  6.58it/s]                   

Tiempo de ejecución: 23.55 segundos


**Exportar archivos de datos con embeddings generados (en formato CSV):**

In [27]:
# Exportar archivos de datos con embeddings en CSV por cada tema
datasetEmbeddingsCorrupcion.to_csv(f"{nombreArchivoCorrupcion}Embeddings.csv")
datasetEmbeddingsNoDiscriminacion.to_csv(f"{nombreArchivoNoDiscriminacion}Embeddings.csv")
datasetEmbeddingsEduacion.to_csv(f"{nombreArchivoEducacion}Embeddings.csv")
datasetEmbeddingsSalud.to_csv(f"{nombreArchivoSalud}Embeddings.csv")
datasetEmbeddingsTransparencia.to_csv(f"{nombreArchivoTransparencia}Embeddings.csv")
datasetEmbeddingsViolenciaMujeres.to_csv(f"{nombreArchivoViolenciaMujeres}Embeddings.csv")	

print(f"{nombreArchivoCorrupcion}Embeddings Descargado en csv")
print(f"{nombreArchivoNoDiscriminacion}Embeddings Descargado en csv")
print(f"{nombreArchivoEducacion}Embeddings Descargado en csv")
print(f"{nombreArchivoSalud}Embeddings Descargado en csv")
print(f"{nombreArchivoTransparencia}Embeddings Descargado en csv")
print(f"{nombreArchivoViolenciaMujeres}Embeddings Descargado en csv")

20240325_1759_PreguntasDebateINE_MuestraEstratificada_CorrupcionEmbeddings Descargado en csv
20240325_1759_PreguntasDebateINE_MuestraEstratificada_NoDiscriminacionEmbeddings Descargado en csv
20240325_1759_PreguntasDebateINE_MuestraEstratificada_EducacionEmbeddings Descargado en csv
20240325_1759_PreguntasDebateINE_MuestraEstratificada_SaludEmbeddings Descargado en csv
20240325_1759_PreguntasDebateINE_MuestraEstratificada_TransparenciaEmbeddings Descargado en csv
20240325_1759_PreguntasDebateINE_MuestraEstratificada_ViolenciaMujeresEmbeddings Descargado en csv


## 3. Referencias:

* Instituto Nacional Electoral. (2024). Acuerdo INE/CG95/2024. Acuerdo del Consejo General del Instituto Nacional Electoral por el que se define la metodología, así como la instancia que seleccionará las preguntas provenientes de redes sociales relativas al Formato Tipo A que se utilizará en el Primer Debate Presidencial en el Proceso Electoral Federal 2023-2024. Repositorio Documental INE. https://repositoriodocumental.ine.mx/xmlui/bitstream/handle/123456789/164296/CGex202402-08-ap-3.pdf
* Instituto Nacional Electoral. (2024). Anexo I. Metodología Selección de Preguntas para Debate Formato A. Repositorio Documental INE. https://repositoriodocumental.ine.mx/xmlui/bitstream/handle/123456789/164296/CGex202402-08-ap-3-a1.pdf
* Reimers, N., & Gurevych, I. (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks (arXiv:1908.10084). arXiv. http://arxiv.org/abs/1908.10084
* Wang, L., Yang, N., Huang, X., Yang, L., Majumder, R., & Wei, F. (2024). Multilingual E5 Text Embeddings: A Technical Report. arXiv preprint arXiv:2402.05672. Recuperado de https://arxiv.org/abs/2402.05672